## Motherfuckers


In [14]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Python standard library imports
from typing import Self, Any
from random import sample

from numpy import ndarray

from matplotlib.pyplot import subplots, show

from keras.datasets.cifar10 import load_data as load_cifar10

# model building imports
from keras import Model, Sequential, Input

from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.layers import LeakyReLU
from keras.ops import add
from keras.utils import to_categorical

# model training imports
from keras.optimizers import SGD
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy, AUC, F1Score
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler

import pandas as pd

from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

from keras.applications import VGG16
from keras.models import Model
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from keras.optimizers import SGD

from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from datetime import datetime

In [7]:
metadata = pd.read_csv('../data/rare_species 1/metadata.csv')
metadata.head(10)

rare_species_id  eol_content_id  eol_page_id  \
0  75fd91cb-2881-41cd-88e6-de451e8b60e2        12853737       449393   
1  28c508bc-63ff-4e60-9c8f-1934367e1528        20969394       793083   
2  00372441-588c-4af8-9665-29bee20822c0        28895411       319982   
3  29cc6040-6af2-49ee-86ec-ab7d89793828        29658536     45510188   
4  94004bff-3a33-4758-8125-bf72e6e57eab        21252576      7250886   
5  dc48f2ce-4feb-4ef7-b2a2-c3c3f42bf19b        28657539       491832   
6  3d881320-8ba8-4580-a72c-0e7ab116b664        29548208     47043290   
7  7faca96a-54e6-4c80-b9e4-77ab126d904a        21232818      1033999   
8  9f89ecab-aabd-41a4-b5b4-8ce106d85959        20315204     46561012   
9  b6ec7a70-c470-4ede-8930-05844e1efd2e        20124498     46570095   

    kingdom      phylum            family  \
0  animalia    mollusca         unionidae   
1  animalia    chordata       geoemydidae   
2  animalia    chordata  cryptobranchidae   
3  animalia    chordata          turdidae   
4  animalia    chordata         indriidae   
5  animalia  arthropoda        formicidae   
6  animalia    chordata      fringillidae   
7  animalia  arthropoda         gomphidae   
8  animalia    chordata      myliobatidae   
9  animalia    chordata    pleuronectidae   

                                           file_path  
0  mollusca_unionidae/12853737_449393_eol-full-si...  
1  chordata_geoemydidae/20969394_793083_eol-full-...  
2  chordata_cryptobranchidae/28895411_319982_eol-...  
3  chordata_turdidae/29658536_45510188_eol-full-s...  
4  chordata_indriidae/21252576_7250886_eol-full-s...  
5  arthropoda_formicidae/28657539_491832_eol-full...  
6  chordata_fringillidae/29548208_47043290_eol-fu...  
7  arthropoda_gomphidae/21232818_1033999_eol-full...  
8  chordata_myliobatidae/20315204_46561012_eol-fu...  
9  chordata_pleuronectidae/20124498_46570095_eol-...

In [9]:
# Base folder where images live
base_dir = Path("../data/rare_species 1")

# Load metadata
metadata = pd.read_csv(base_dir / "metadata.csv")

# Add full image path
metadata["full_path"] = metadata["file_path"].apply(lambda x: base_dir / x)

# Optional: remove missing files (if any)
metadata = metadata[metadata["full_path"].apply(lambda x: x.exists())]

# Encode family as categorical labels
metadata["label"] = metadata["family"].astype("category").cat.codes
label_to_family = dict(enumerate(metadata["family"].astype("category").cat.categories))

# Split: Train (70%), Val (15%), Test (15%)
train_df, temp_df = train_test_split(metadata, stratify=metadata["label"], test_size=0.30, random_state=42)
val_df, test_df = train_test_split(temp_df, stratify=temp_df["label"], test_size=0.50, random_state=42)

print("Train:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))

Train: 8388 Val: 1797 Test: 1798


In [11]:
# Image size expected by VGG16
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Helper function: load, resize, and rescale
def preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)  # In case images are in JPEG format
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32) / 255.0  # Rescale to [0, 1]
    return image, label

# Function to create tf.data.Dataset from dataframe
def create_dataset(df):
    paths = df["full_path"].astype(str).values
    labels = df["label"].values
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

# Create datasets
train_ds = create_dataset(train_df)
val_ds = create_dataset(val_df)
test_ds = create_dataset(test_df)

In [13]:
# Get number of families (output classes)
num_classes = metadata["label"].nunique()

# Input shape must match resized images
input_shape = (224, 224, 3)
inputs = Input(shape=input_shape)

# Load VGG16 backbone (without top classifier), freeze its weights
base_model = VGG16(include_top=False, weights="imagenet", input_tensor=inputs)
base_model.trainable = False  # Freeze feature extractor for now

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)  # Optional: prevent overfitting
outputs = Dense(num_classes, activation="softmax")(x)

# Build final model
model = Model(inputs=inputs, outputs=outputs)

# Compile
model.compile(
    optimizer=SGD(learning_rate=0.01, momentum=0.9),
    loss="sparse_categorical_crossentropy",  # because labels are integers
    metrics=["accuracy"]
)

model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 202)            │       103,626 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,818,314 (56.53 MB)

 Trainable params: 103,626 (404.79 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [18]:
# Create an output directory for saving logs/models
output_dir = Path("./outputs")
output_dir.mkdir(exist_ok=True)

# Timestamp to uniquely name files
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

# Callbacks
callbacks = [
    ModelCheckpoint(
        filepath=output_dir / f"vgg16_baseline_best_{timestamp}.keras",
        save_best_only=True,
        monitor="val_accuracy",
        mode="max",
        verbose=1
    ),
    CSVLogger(output_dir / f"training_log_{timestamp}.csv"),
    EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True)
]

# Fit the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
    callbacks=callbacks
)

Epoch 1/3


KeyboardInterrupt: 